# 导言

 ```{epigraph}
 When I approach a programming problem, I first think about the types and abstractions that will be useful; I think about statically enforcing behaviour; and I don’t worry about the cost of intermediary data structures, because that price is almost never paid in full. 
 
 --Michael Walker
 ```

这一部分是关于信号处理技术的工程实现。

工程实现依赖于有效的单元操作。单元操作分两大类：其一是算法的单元操作；其二是编程技巧的单元操作。

算法的单元操作也分两类，第一类是基本常见的单元操作，比如`shift`，`decimatioin`；另外一类是算法本身的单元操作，比如`lct`中的折叠算子。从某种意义上说，一个算法就是一个单元操作的集合与串通这些单元操作的逻辑。所以，算法上对单元操作的思考和掌控一般都比较严密，也因此比较到位。

编程技巧上对单元操作的梳理也是比较丰富的，遗憾的是由于只要动手，一般都会比较快地得到结果，因此在单元操作上的设计比较容易被忽视，这是我们在工作中需要重视的。

在大的框架上，编程技巧有以下几类：
1. Python data model
2. Object oriented programming (oop)
   - 抽象类（abstract class）与具体类（concrete class）
   - traits和mixin
3. 对象的可变更性（mutable）与不可变更性（immutable）
4. 函数式（functional programming）
   
更底层的技巧依赖于数据结构的经验，常用的有二叉树等。

## Python collections 与 data model
这段资料取自于{cite}`ramalho2015fluent`的第一章。这里简单介绍`collections`和data model，是我们常用的工具。

### collections

In [1]:
import collections

Card=collections.namedtuple('Card', ['rank', 'suit'])

In [2]:
Card('2','spades')

Card(rank='2', suit='spades')

## data model
我们用一个模拟扑克牌的案例来说明data model的用法，定义一个类叫`FrenchDeck`：

In [3]:
class FrenchDeck:
    ranks = [str(n) for n in range(2,11)]+list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards=[Card(rank,suit) for suit in self.suits
                                    for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self,position):
        return self._cards[position]

`FrenchDeck`比较简单，但功能强大。这是因为用了特殊函数`__len__`和`__getitem__`。下面来说明其主要功能。

### `__len__`
`__len__`的功能是返回纸牌的个数。

`FrenchDeck`中，唯一一个成员就是`_cards`，用`list comprehension`的方法（以上程序第6行）生成，因此是collection中的一个类。`__len__`是应用`len`函数于这个成员，算出成员里面元素的个数，也就是纸牌的个数。

In [4]:
deck=FrenchDeck()
len(deck)

52

当函数`len`作用于`deck`，系统中是委托`FrenchDeck`中的特殊函数`__len__`来实现。

### `__getitem__`
定义这个特殊类方法使得`FrenchDeck`具备以下功能：
1. 支持slicing：这是因为我们在`__getitem__`中委托算子`[]`进行对类成员`_cards`的指针操作。因为`_cards`的类型是`list`，这种简练的委托，使得`FrenchDeck`变成了像`list`数据结构。
2. 使得`FrenchDeck`成为*iterable*，这样许多python中适用于*iterable*的程序库能被应用于`FrenchDeck`。

值得强调的是，`FrenchDeck`的实现不是通过继承的方式，更多的是一种组合的方式来实现。这是*data model*非常重要的特征。

#### 类`list`
抽取第`n`张纸牌，`n`的方式与`list`一样，如下`deck[0]`为第一张纸牌，`deck[-1]`为最后一张纸牌：

In [5]:
deck[0]

Card(rank='2', suit='spades')

In [6]:
deck[-1]

Card(rank='A', suit='hearts')

通过调用`random.choice`，使得`FrenchDeck`支持随机抽取纸牌的功能：

In [7]:
from random import choice

choice(deck)

Card(rank='10', suit='diamonds')

支持*slicing*:

In [8]:
deck[0:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [9]:
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

#### *iterable*

In [10]:
for i,j in zip(range(len(deck)),deck):
    if i>4:
        break
    print(j)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')


In [11]:
for i,j in zip(range(len(deck)),reversed(deck)):
    if (i<4):
        print(j)
    else:
        break

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')


In [12]:
Card('Q', 'hearts') in deck

True

我们来看看排序功能：

In [13]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [14]:
for i,card in zip(range(len(deck)),sorted(deck, key=spades_high)):
    if i<6:
        print(card)
    else:
        break

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')


## traits和mixin的案例
这个案例来自于文章[Higher-Order Mixin Classes, or "Traits, but not quite!"](https://web.archive.org/web/20161009062141/http://stupid-python-tricks.blogspot.com/2015/04/computed-properties-and-higher-order.html)，所用到的技巧非常漂亮。

对于颜色，定义一个trait，通过函数的形式来生成trait类。用到匿名类`Inner`，配合`setattr`以及`property`两个函数的使用来设定匿名类的参数。

In [15]:
def HasColor(name, default=None, doc=None):
    """
    Return an anonymous class that has a color property
    named `name`, with a default value of `default` and
    a docstring of `doc`.
    """

    propname = "__color_{}".format(name)
    colors = {
        "black": 0x00000000,
        "white": 0x00ffffff,
        "red":   0x00ff0000,
        "green": 0x0000ff00,
        "blue":  0x000000ff
    }

    def getter(self):
        return getattr(self, propname, default)

    def setter(self, value):
        if isinstance(value, int):
            if isinstance(value, bool):
                raise TypeError("integer colors must be ints")

            if value < 0 or value > 0x00ffffff:
                raise ValueError("integer colors must be 24-bit")

            setattr(self, propname, value)

        elif isinstance(value, str):
            if value not in colors:
                raise ValueError("unknown color '{}'".format(value))

            setattr(self, propname, colors[value])

        else:
            raise TypeError("color specifications must be ints or strs")


    class Inner:
        pass

    setattr(Inner, name, property(getter, setter, None, doc))
    return Inner

使用trait来组装下面的`Canvas`和`Button`，这样的组装方式显得非常灵活，合理，这就是mixin的概念：

In [16]:
class GraphicsObject: # The base class of our hierarchy.
    pass

class Canvas(GraphicsObject,
            HasColor("foreground"),
            HasColor("background")):
    pass

class Button(Canvas, # Inherit Canvas's foreground and background.
             HasColor("border"),
             HasColor("text")):
    pass

In [17]:
c = Canvas()
c.foreground = 'red'

b = Button()
b.foreground = 'black'
b.text = 'green'


In [18]:
print("canvas foreground:",hex(c.foreground))
print("button foreground:",hex(b.foreground))
print("button text:",hex(b.text))

canvas foreground: 0xff0000
button foreground: 0x0
button text: 0xff00


## 不可变更（immutable）类
### 类的隐私参数
类的隐私参数通过在参数名加前缀`__`来实现：

In [19]:
class foo:
    def __init__(self,v):
        self.__v=v

    def __repr__(self):
        return str(self.__v)
f1=foo(3)

In [20]:
#print(f1.__v)会产生错误：AttributeError: 'foo' object has no attribute '__v'

In [21]:
print(f1)

3


### 如何实现类的不可变更性
通过屏蔽掉类的`__setattr__`和`__delattr__`特殊函数，我们可以达到准不可变更类。下面的案例分析，从中可以看到我们为什么用”准“这个词。

In [22]:
import inspect

class Immutable(object):
    """Inherit this class to make the child class immutable"""
    def __setattr__(self, *args):
        if inspect.stack()[1][3] == '__init__':
            object.__setattr__(self, *args)
        else:
            raise TypeError('Cannot modify Immutable instance')

    __delattr__=__setattr__

In [23]:
class foo(Immutable):
    def __init__(self,data):
        self.__data=data
    
    def __repr__(self):
        return str(id(self.__data))
    
    def __getitem__(self,p):
        return self.__data[p]
    
    def __setitem__(self,p,v):
        self.__data[p]=v

In [24]:
foo1=list(range(4))
foo2=foo(foo1)
print(foo2,id(foo1))

140268132462272 140268132462272


In [25]:
foo2[1]='s'
for i in foo2:
    print(i)

0
s
2
3


In [26]:
for i in foo1:
    print(i)

0
s
2
3


这里我们看到以下非常重要的几点：
1. 数组作为类构造函数的输入参数是按pass by reference来进行，这里的foo1就是被pass by reference，因为foo1的`id`值跟`foo`里面`__data`的`id`值是一样的；同时我们也看到`foo2`中`__data[1]`改成字符`s`后，`foo1[1]`也相应改成了`s`。
2. 虽然类`foo`继承了`Immutable`，屏蔽了对类参数的更改，这只是限制了对`__data`的更改，并不能限制`__data`内容的更改，也就是对参数的更改与对参数内容的更改是两个不同的概念。
3. 真正能够让一个继承`Immutable`类成为绝对不可更改，必须要求类参数本身也是不可更改的类。
4. 在没有声称`__setitem__`的情况下，作为`Immutable`的子类，外部对`foo`确实是没有更改的途径。对`foo`内部参数`__data`内容的更改只能在`foo`内部进行，这在一定程度上实现了不可更改性。

## 常用的技巧
1. map
2. itertools
3. lambda
4. exception
5. decorator
6. closure
7. generator
8. threading
9. regular expression

### itertools
我们来看看`itertools`的功能，这个程序包功能强大，对编程的方式影响深远。在这里我们探讨的主要功能是`iter`，`islice`，`repeat`和`starmap`，材料主要来源于
[What are the uses of iter(callable, sentinel)?](https://stackoverflow.com/questions/38087427/what-are-the-uses-of-itercallable-sentinel)。至于其他功能和一些基础材料，可参见[Itertools in Python 3, By Example](https://realpython.com/python-itertools/)。

In [27]:
from itertools import starmap, islice, repeat, takewhile, tee, count

In [28]:
n=6
N=23
M=1+int(N/n)
list_obj=list(range(N))

#### itertools.iter和itertools.islice

假设有一个序列：$0,1,2,\cdots,N-1,\quad N>0$，给定一个正整数`n`，我们需要一个分组的算法，将这组数按序分割成$M$组，每一组有`n`个数（最后一组的个数可能小于`n`）。

In [29]:
import time

下面的代码建立在以下两点的基础上：
1. 一系列的`islice`作用于*同一个对象*。
2. 这个对象是被分析序列`list_obj`的一个`iter`。

这就保证第一次执行`islice`，我们截取出第一组`n`个数，第二次运行，截取第二组，余此类推。

In [30]:
#%timeit -r1 time.sleep(.001)
%time
tmp=iter(list_obj)
for i in range(M):
    print(tuple(islice(tmp,n)))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs
(0, 1, 2, 3, 4, 5)
(6, 7, 8, 9, 10, 11)
(12, 13, 14, 15, 16, 17)
(18, 19, 20, 21, 22)


`islice`这种滑动的性质是建立在被作用的对象是被分析序列`list_obj`的一个`iter`。为了说明这个特征，下面的案例是将`islice`直接作用于被分析序列上，因为没有`iter`的特征，我们看到滑动的性质消失了：

In [31]:
print(list(islice(list_obj,n)),list(islice(list_obj,n)))

[0, 1, 2, 3, 4, 5] [0, 1, 2, 3, 4, 5]


用`islice`结合`iter`来获取滑动的性质，这种技巧非常重要。在这个技巧下，上面的算法的本质如下：
1. 产生一个函数系列，序列中的元素`islice`作用于同一个`iter(list_obj)`对象，
2. 按序来执行这个函数序列从而达到分段截取的目的。

先来看看用list comprehension来产生这个函数序列：

In [32]:
#%timeit -r1 time.sleep(.001)
tmp=iter(list_obj)
foo=[islice(tmp,n) for i in range(M)]
for i in range(M):
    print(tuple(foo[i]))

(0, 1, 2, 3, 4, 5)
(6, 7, 8, 9, 10, 11)
(12, 13, 14, 15, 16, 17)
(18, 19, 20, 21, 22)


将算法封装成函数，返回函数序列的`iter`，利用这个`iter`可以轻松实现上面所说的第二步。这里我们用到了`iter`中的一个功能，我们可以将一个`callable`塞到`iter`里，让`iter`按这个`callable`来滑动，同时给定一个滑动终止的状态。我们所塞给`iter`的是`map`的特殊函数`__next__`，终止的状态是`()`，这是因为我们通过`map`将`islice`的结果转换成`tuple`，当被分析序列没有数据点的时候，`islice`返回空值，而`tuple`作用于空值为`()`，这个时候分割过程结束。

In [33]:
def grouper_v1(n,m,iterable):
    tmp=iter(iterable)
    s=[islice(tmp,n) for i in range(m)]
    return iter(map(tuple,s).__next__,())

In [34]:
#%timeit -r1 time.sleep(.001)
foo=grouper_v1(n,M,list_obj)
for i in foo:
    print(i)

(0, 1, 2, 3, 4, 5)
(6, 7, 8, 9, 10, 11)
(12, 13, 14, 15, 16, 17)
(18, 19, 20, 21, 22)


### repeat
`repeat`将一个输入对象（第一个输入参数）重复`n`次（第二个输入参数），返回所产生的序列的一个`iter`。其最大的特征是同一个对象重复了`n`次，见下例：

In [35]:
foo=repeat(iter(list_obj),2)
for i in foo:
    print(id(i))

140268176109104
140268176109104


跟list comprehension比较，我们可以看到在list comprehension里，`iter(list_obj)`不断产生新的对象。这就是为什么在`grouper_v1`中，我们用中间变量`tmp`来固定住对象。

In [36]:
foo=[iter(list_obj) for i in range(2)]
for i in range(2):
    print(id(foo[i]))

140268176102528
140268176107568


从上面的`for in foo`与下面的例子，我们看到`repeat`返回的是一个`iter`。

In [37]:
foo=repeat(iter(list_obj),2)
for i in range(2):
    print(list(next(foo)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
[]


### starmap
我们可以利用`repeat`来产生`islice`的输入参数，再用`starmap`来生成函数序列。这里不能用`map`，因为`islice`的输入是两个参数。由此我们产生函数序列的工具箱又多了两工具，`starmap`和`repeat`。

In [38]:
foo=starmap(islice,repeat((iter(list_obj),n)))
for i in range(M):
    print(list(next(foo)))

[0, 1, 2, 3, 4, 5]
[6, 7, 8, 9, 10, 11]
[12, 13, 14, 15, 16, 17]
[18, 19, 20, 21, 22]


有了函数序列，剩下来的事跟前面是一样的，将这个算法封装如下：

In [39]:
def grouper(n, iterable):
    '''Returns a generator yielding n sized tuples from iterable

    For iterables not evenly divisible by n, the final group will be undersized.
    '''
    # Keep islicing n items and converting to groups until we hit an empty slice
    return iter(map(tuple, starmap(islice, repeat((iter(iterable),n)))).__next__,())

In [40]:
#%timeit -r1 time.sleep(.001)
%time
foo=grouper(n,list_obj)
for i in foo:
    print(i)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs
(0, 1, 2, 3, 4, 5)
(6, 7, 8, 9, 10, 11)
(12, 13, 14, 15, 16, 17)
(18, 19, 20, 21, 22)


上面的两个算法的差异在于后者是**lazy**，而前者不是。前者在提取结果前，函数序列是已经建立起来，而后者在去算第`n`步时，`islice`才产生。所以后者更合理。这个差异在这个案例体现不强，因为`islice`作用于同一个对象，当数据不是特别大的时候，问题不大。

### takewhile
我们可以用`takewhile`代替`iter`，用`bool`函数作为predicate（`takewhile`的第一个输入参数），用函数序列被访问的序列，也就是另外一个输入参数。当函数序列返回`()`，`bool`函数返回`False`，`takewhile`结束。

In [41]:
%time
foo=takewhile(bool,map(tuple, starmap(islice, repeat((iter(list_obj),n)))))
for i in foo:
    print(i)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs
(0, 1, 2, 3, 4, 5)
(6, 7, 8, 9, 10, 11)
(12, 13, 14, 15, 16, 17)
(18, 19, 20, 21, 22)


In [42]:
foo=takewhile(bool,map(tuple, starmap(islice, repeat((iter(list_obj),n)))))
for i in foo:
    print(bool(i))

True
True
True
True


In [43]:
bool(())

False

In [44]:
foo=iter(list_obj)
for i in range(2):
    print(list(islice(foo,3,6)))

[3, 4, 5]
[9, 10, 11]


## Decorator
In Python, a [decorator](https://peps.python.org/pep-0318/) is the implementation of a pattern that allows adding a behavior to a function or a class. It is usually expressed with the @decorator syntax prefixing a function. Here’s a contrived example:

In [45]:
def some_decorator(f):
    def wraps(*args):
        print(f"Calling function '{f.__name__}'")
        return f(args)
    return wraps

@some_decorator
def decorated_function(x):
    print(f"With argument '{x}'")

In [46]:
decorated_function('Python')

Calling function 'decorated_function'
With argument '('Python',)'


## Closure
A [closure](https://en.wikipedia.org/wiki/Closure_%28computer_programming%29) is a function where every free variable, everything except parameters, used in that function is bound to a specific value defined in the enclosing scope of that function. In effect, closures define the environment in which they run, and so can be called from anywhere.

The concepts of lambdas and closures are not necessarily related, although lambda functions can be closures in the same way that normal functions can also be closures. Some languages have special constructs for closure or lambda (for example, Groovy with an anonymous block of code as Closure object), or a lambda expression (for example, Java Lambda expression with a limited option for closure).

Here’s a closure constructed with a normal Python function:

In [47]:
def outer_func(x):
    y = 4
    def inner_func(z):
        print(f"x = {x}, y = {y}, z = {z}")
        return x + y + z
    return inner_func

for i in range(3):
    closure = outer_func(i)
    print(f"closure({i+5}) = {closure(i+5)}")

x = 0, y = 4, z = 5
closure(5) = 9
x = 1, y = 4, z = 6
closure(6) = 11
x = 2, y = 4, z = 7
closure(7) = 13


## 环境管理器

### 基本概念
这里的材料来自于[Context Manager Using @contextmanager Decorator](https://www.geeksforgeeks.org/context-manager-using-contextmanager-decorator/?ref=lbp)。

In [48]:
# Python program creating a
# context manager

class ContextManager():
	def __init__(self):
		print('init method called')
		
	def __enter__(self):
		print('enter method called')
		return self
	
	def __exit__(self, exc_type, exc_value, exc_traceback):
		print('exit method called')

# Driver code
with ContextManager() as manager:
	print('with statement block')

init method called
enter method called
with statement block
exit method called


In the above code, __enter__ will be executed when control enters with and __exit__, when control leaves with clause. We can simply make any function as a context manager with the help of contextlib.contextmanager decorator without having to write a separate class or __enter__ and __exit__ functions.
 

Using @contextmanager
We have to use contextlib.contextmanager to decorate a generator function which yields **exactly once**. Everything before yield is considered to be __enter__ section and everything after, to be __exit__ section. The generator function should yield the resource. 

In [49]:
# Python program for creating a
# context manager using @contextmanager
# decorator

from contextlib import contextmanager

@contextmanager
def ContextManager():
	
	# Before yield as the enter method
	print("Enter method called")
	yield
	
	# After yield as the exit method
	print("Exit method called")

with ContextManager() as manager:
	print('with statement block')


Enter method called
with statement block
Exit method called


我们来看看一个简单的案例，对文件写入信息。这里的材料来自于[with statement in Python](https://www.geeksforgeeks.org/with-statement-in-python/)。

### 类方式实现

In [50]:
# a simple file writer object
 
class MessageWriter(object):
    def __init__(self, file_name):
        self.file_name = file_name
     
    def __enter__(self):
        self.file = open(self.file_name, 'w')
        return self.file
 
    def __exit__(self, *args):
        self.file.close()
 
# using with statement with MessageWriter
 
with MessageWriter('my_file.txt') as xfile:
    xfile.write('hello world')

### 生成器与环境管理器

A class based context manager as shown above is not the only way to support the with statement in user defined objects. The contextlib module provides a few more abstractions built upon the basic context manager interface. Here is how we can rewrite the context manager for the MessageWriter object using the contextlib module. 

In [51]:
from contextlib import contextmanager


class MessageWriter(object):
	def __init__(self, filename):
		self.file_name = filename

	@contextmanager
	def open_file(self):
		try:
			file = open(self.file_name, 'w')
			yield file
		finally:
			file.close()


# usage
message_writer = MessageWriter('hello.txt')
with message_writer.open_file() as my_file:
	my_file.write('hello world')


In this code example, because of the [yield](https://www.geeksforgeeks.org/use-yield-keyword-instead-return-keyword-python/) statement in its definition, the function open_file() is a [generator function](https://www.geeksforgeeks.org/generators-in-python/). When this open_file() function is called, it creates a resource descriptor named file. This resource descriptor is then passed to the caller and is represented here by the variable my_file. After the code inside the with block is executed the program control returns back to the open_file() function. The open_file() function resumes its execution and executes the code following the yield statement. This part of code which appears after the yield statement releases the acquired resources. The @contextmanager here is a [decorator](https://www.geeksforgeeks.org/decorators-in-python/). The previous class-based implementation and this generator-based implementation of context managers is internally the same. While the later seems more readable, it requires the knowledge of generators, decorators and yield.

## tail recursive

## pattern matching

## partial functions

## typing

## 函数式
1. 容器（container）：对特定对象的一个封装。用信号的作为例子，信号是关于数组中的元素的一个容器。
2. functor：一种特殊的容器，其中定义了`fmap`函数，其作用是应用外部定义的函数`f`（也就是`fmap`的输入参数）作用于被封装的类，并且将作用的结果用同样的容器封装起来作为`fmap`的输出。假设被封装的对象是类型`A`，`f`作用于对象的结果类型是`B`，即，`f: A->B`，那么`fmap: f->F[B]`，`F`是原来functor的类型，`F[B]`指的是含有类型`B`的functor，类型与原来的`F`一样，只是所含的元素是`B`类型（原来是`A`类型）。
3. monad：在functor的基础上加上`bind`函数。用`M`代表一个monad的类型，跟`fmap`不同在于，`bind`要求输入参数`f`为`A->M[B]`的函数，`bind`的返回值为`M[B]`类型，而且是直接返回`f`的返回值。


### 一个monad的案例

In [52]:
from typing import Generic, TypeVar, Callable
from immutable import Immutable

A = TypeVar("A")
B = TypeVar("B")

class Maybe(Generic[A]):
    pass

class Just(Maybe[A],Immutable):

    def __init__(self, val: A):
        self.val = val

    def fmap(self, func: Callable[[A], B]) -> Maybe[B]:
        try:
            new_val = func(self.val)
            return Just(new_val)
        except Exception as e:
            return Nothing()

    def bind(self, func: Callable[[A], Maybe[B]]) -> Maybe[B]:
        try:
            new_val = func(self.val)
            return new_val
        except Exception as e:
            return Nothing()
    
    def __repr__(self):
        return str(self.val)

class Nothing(Maybe[A]):

    def fmap(self, func: Callable[[A], B]) -> Maybe[B]:
        return self
    
    def bind(self, func: Callable[[A], Maybe[B]]) -> Maybe[B]:
        return self
    
    def __repr__(self):
        return "Nothing...."

In [53]:
Just("123").fmap(float)

123.0

In [54]:
Just("123a").fmap(float)

Nothing....

In [55]:
Just(" -10 ").fmap(int).fmap(abs)

10

In [56]:
def decrease(num: int) -> Maybe[int]:
   if num >= 1:
       return Just(num-1)
   else:
       return Nothing()

In [57]:
Just(5).bind(decrease)

4

In [58]:
Just(0).bind(decrease)

Nothing....

In [59]:
Just("-10").fmap(int).fmap(float).fmap(abs).fmap(str)

10.0

In [60]:
Nothing().bind(lambda x: Just(x))

Nothing....

:::{note}
This text is **standard** *Markdown*
:::

## 常用的数据结构

```{bibliography}
:filter: docname in docnames
```